## Trande-Offs in Missing Data Conventions

## Missing Data in Pandas
using sentinel value to indicate missing values

### `None`:Pythobic missing data

In [1]:
import numpy as np
import pandas as pd

In [3]:
vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

This `dtype=object` means that any operations on the data will be done at the Python level, with much more overhead than the typically fast operation

In [6]:
for dtype in ['object', 'int']:
    print('dtype= {}'.format(dtype))
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype= object
134 ms ± 380 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype= int
4.47 ms ± 51.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



The use of Python objects in an array also means that if you perform aggregations like `sum()` or `min()` access an array with a `None` value, you will generally get an error

In [7]:
vals1.min()

TypeError: '<=' not supported between instances of 'int' and 'NoneType'

### `NaN` : Missing numerical data

`NaN` is a special floating-point value recognized by all systems.  
This means it supports fast operations pushed into compiled code.  
Regardless of the operation, the result of arithmetic with `NaN` will be another `NaN`


In [11]:
vals2 = np.array([1, np.nan, 3, 4])
vals2.dtype

dtype('float64')

In [12]:
1 +np.nan

nan

In [13]:
0*np.nan

nan

In [14]:
vals2.sum()

nan

Meanwhile Numpy does provide some special aggregations that will ignore these missing data




In [16]:
# it will ignore the missing data of NaN
np.nansum(vals2)


8.0

### `NaN` and `None` inPandas
Pandas is built to handle the two of them nearly interchangeably, converting between them where appropriate

For types that don't have an available sentinel value, Pandas automatically type-casts when NaN values are present.

In [18]:
pd.Series([1,2,3,None])

0    1.0
1    2.0
2    3.0
3    NaN
dtype: float64

In [19]:
x = pd.Series(range(2), dtype='int')
x

0    0
1    1
dtype: int32

In [23]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

The following table lists the upcasting conventions in Pandas when NA values are introduced:

|Typeclass     | Conversion When Storing NAs | NA Sentinel Value      |
|--------------|-----------------------------|------------------------|
| ``floating`` | No change                   | ``np.nan``             |
| ``object``   | No change                   | ``None`` or ``np.nan`` |
| ``integer``  | Cast to ``float64``         | ``np.nan``             |
| ``boolean``  | Cast to ``object``          | ``None`` or ``np.nan`` |

Keep in mind that in Pandas, string data is always stored with an ``object`` dtype.

## Operation on Null Values

Detecting, removing, and replacing null values in Pandas data structures.
+ `isnull()`:Generate a boolean mask indicating missing data
+ `notnull()`:Opposite of `isnull()`
+ `dropna()`:Return a filtered version of the data
+ `fillna()`:Return a copy of the data with missing filled or imputed

### Detecting null values

In [26]:
# structuring a Pandas Series
data = pd.Series([1, np.nan, 'hello', None])

data

0        1
1      NaN
2    hello
3     None
dtype: object

In [33]:
# Generate a boolean mask indicating missing value
print(data.isnull())
print('')

# Boolean masks can be used directly 
# as a Series or DataFrame index
data[data.isnull()]

0    False
1     True
2    False
3     True
dtype: bool



1     NaN
3    None
dtype: object

In [34]:
data[data.notnull()]

0        1
2    hello
dtype: object

### Dropping null values
Return a copy of filled data 

In [39]:
data.dropna()


0        1
2    hello
dtype: object

In [40]:
data

0        1
1      NaN
2    hello
3     None
dtype: object

For a Dataframe, there are more options


In [41]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


By default, `dropna()` will drop all rows in which **any** null value is present

In [42]:
df.dropna()

,0,1,2
1,2.0,3.0,5


Alternatively, we can drop `NaN` values along a different axis;  
`axis=1` drops all columns containings a null values

In [43]:
df.dropna(axis=1)

,2
0,2
1,5
2,6


But this drops some good data as well.  
If we are just interestted in dropping rows or columns with `all` NaN values, or a majority of NA values  
This can be specified through the `how` or `thresh` parameters, which allow fine control of the number os nulls to allow through

The default is `how=any`

In [45]:
df[3] = np.nan

In [46]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [48]:
df.dropna(axis=1, how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [49]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


For finer-grained control, the `thresh` parameter lets us specify a **minimum** number of **non-null** value for the row/column to be kept

In [55]:
df.dropna(axis=1, thresh=3)

,2
0,2
1,5
2,6


### Filling null values
`fillna()`  
Return a copy of the array with the null values replaced

In [57]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [59]:
# fill NaN entries with zero
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

We can specify a **forward-fill(back-fill)** to propagate the previous value forward(backward)

In [61]:
# forward-fill
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [62]:
# backward fill
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

For DataFrame's, the options are similar, but we can also specify an `axis` along which the fills take place

In [63]:
df


,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [64]:
df.fillna(0)

,0,1,2,3
0,1.0,0.0,2,0.0
1,2.0,3.0,5,0.0
2,0.0,4.0,6,0.0


In [67]:
df.fillna(axis=0, method='ffill')


,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,2.0,4.0,6,NaN


In [69]:
df.fillna(axis=1, method='ffill')

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0
